In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)
import joblib

In [4]:
df = pd.read_csv("telecom_churn.csv")
print("Dataset cargado correctamente:", df.shape)

X = df.drop("Churn", axis=1)
y = df["Churn"]


Dataset cargado correctamente: (3333, 11)


In [10]:
# Escalar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)


In [16]:
#Entrenar el modelo
def build_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

#Calcula las métricas principales de evaluación
def summarize_classification(y_true, y_pred, y_prob):
    return {
        "Accuracy": round(accuracy_score(y_true, y_pred), 4),
        "Precision": round(precision_score(y_true, y_pred), 4),
        "Recall": round(recall_score(y_true, y_pred), 4),
        "F1": round(f1_score(y_true, y_pred), 4),
        "ROC-AUC": round(roc_auc_score(y_true, y_prob), 4)
    }


In [28]:
# Entrenamiento de múltiples modelos
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver="liblinear"),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    print(f"\n Entrenando el modelo: {name}")
    trained = build_model(model, X_train, y_train)
    y_pred = trained.predict(X_test)
    y_prob = trained.predict_proba(X_test)[:, 1]
    metrics = summarize_classification(y_test, y_pred, y_prob)
    metrics["Modelo"] = name
    results.append(metrics)

# Convertir resultados a DataFrame
df_results = pd.DataFrame(results)
df_results = df_results[["Modelo", "Accuracy", "Precision", "Recall", "F1", "ROC-AUC"]]
print("\nResultados comparativos:\n")
print(df_results)


 Entrenando el modelo: Logistic Regression

 Entrenando el modelo: Random Forest

Resultados comparativos:

                Modelo  Accuracy  Precision  Recall      F1  ROC-AUC
0  Logistic Regression    0.8561     0.5128  0.2062  0.2941   0.8093
1        Random Forest    0.9250     0.8219  0.6186  0.7059   0.8618


Insight:

-El random forest tuvo la mayor precisión (0.82) y mejor F1-score (0.71) en comparación a la regresión logística. Esto indica que el modelo en arboles tiene mejor capacidad de aprendizaje no lineal, captando mejor las relaciones complejas entre las variables.

In [31]:
#Validación cruzada
print("\nValidación cruzada (5 folds):")
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="f1")
    print(f"{name}: F1 promedio = {scores.mean():.4f}")


Validación cruzada (5 folds):
Logistic Regression: F1 promedio = 0.2775
Random Forest: F1 promedio = 0.7521


Insight:

-La validación cruzada demuestra que el modelo de RandomForest tiene un rendimiento alto y estable en distintos subconjuntos del dataset, indicando una buena capacidad de generalización y bajo riego de sobreajuste.

-El modelo de regresión logística tuvo u F1 bajo, los cual confirma que su capacidad para identificar correctamente los casos positivos es limitada.

-El F1 promedio mas alto es de 0.75 en Random Forest, reforzando su ventaja en el entrenamiento simple y logrando un equilibrio sólido entre la precisión y el recall.

In [34]:
#Seleccion del mejor modelo
best_model_name = df_results.sort_values(by="F1", ascending=False).iloc[0]["Modelo"]
print(f"\nEl meejor modelo seleccionado es: {best_model_name}")

best_model = models[best_model_name]
best_model.fit(X_train, y_train)

joblib.dump(best_model, "model_heuristic.pkl")
joblib.dump(scaler, "preprocessor.pkl")
print("Modelo y preprocesador guardados correctamente.")


El meejor modelo seleccionado es: Random Forest
Modelo y preprocesador guardados correctamente.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f703592-4945-45ea-93be-011448982d02' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>